# **HAND WRITTEN DIGITS CLASSIFIER USING AN MLP**

This project aims to develop a multilayer perceptron that is capable of identifying handwritten digits from 0 to 9.

In [4]:
# imports here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
%matplotlib inline